<a href="https://colab.research.google.com/github/SilverFoxMedia-Tesis/IA_02/blob/main/Modelo_Red_Neuronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [94]:
#Lectura de datos
train = pd.read_csv('/dataset_peru_filtrado_v2.csv')
test = pd.read_csv('/dataset_peru_filtrado_v2.csv')

In [95]:
# Convertir a flotantes sin necesidad de reemplazar comas
train['Weight'] = pd.to_numeric(train['Weight'], errors='coerce')
train['Height'] = pd.to_numeric(train['Height'], errors='coerce')

# Convertir la edad a entero
train['Age'] = pd.to_numeric(train['Age'], downcast='integer', errors='coerce')

# Asegurar que 'Gender' y 'PhysicalActivity' sean tratados como cadenas de texto
# Esto puede no ser necesario si ya están correctamente como strings
train['Gender'] = train['Gender'].astype(str)
train['PhysicalActivity'] = train['PhysicalActivity'].astype(str)
train['Diabetes'] = train['Diabetes'].astype(str)

# Opcional: Manejar valores NaN generados por conversión fallida o datos faltantes
train.dropna(subset=['Weight', 'Height', 'Age', 'Gender', 'PhysicalActivity', 'Diabetes'], inplace=True)

In [96]:
train

,Gender,Age,Diabetes,Weight,Height,AbdominalCircumference,PhysicalActivity
0,MALE,39,NO,75.0,171.0,95.6,No
1,FEMALE,22,NO,65.9,158.1,89.5,Slight
2,FEMALE,20,NO,58.3,161.5,82.9,Slight
3,MALE,33,NO,73.8,170.7,93.7,No
4,FEMALE,38,NO,53.9,151.5,74.9,No
...,...,...,...,...,...,...,...
29392,FEMALE,35,NO,66.4,149.6,96.3,Slight
29393,FEMALE,21,NO,60.4,156.4,84.0,No
29394,MALE,36,NO,63.2,161.9,88.7,Moderate
29395,MALE,34,NO,78.7,167.0,89.9,Slight


In [97]:
def calcular_TMB_MIFFLIN(Weight, Height, Age, Gender, PhysicalActivity):

  # Cálculo de TMB según el género
  if Gender == 'MALE':
      partial_Tmb = (10 * Weight) + (6.25 * Height) - (5 * Age) + 5
  else: # Female
      partial_Tmb = (10 * Weight) + (6.25 * Height) - (5 * Age) - 161

  # Multiplicador según el nivel de actividad
  multiplicador = {
      'No': 1.2,
      'Slight': 1.375,
      'Moderate': 1.55,
      'Strong': 1.725,
      'Very_Strong': 1.9
  }

  # Calorías necesarias según el nivel de actividad
  TMB = partial_Tmb * multiplicador[PhysicalActivity]

  return TMB

In [98]:
def calcular_TMB_FAO_ONU(Weight, Age, Gender):

    if Gender == 'MALE':
        if Age >= 0 and Age <= 3:
            TMB = (60.9 * Weight) - 54
        elif Age >= 4 and Age <= 10:
            TMB = (22.7 * Weight) + 495
        elif Age >= 11 and Age <= 18:
            TMB = (17.5 * Weight) + 651
        elif Age >= 19 and Age <= 30:
            TMB = (15.3 * Weight) + 679
        elif Age >= 31 and Age <= 60:
            TMB = (11.6 * Weight) + 879
        else: # Age > 60
            TMB = (13.5 * Weight) + 487
    else: # Gender == 'FEMALE'
        if Age >= 0 and Age <= 3:
            TMB = (61.0 * Weight) - 51
        elif Age >= 4 and Age <= 10:
            TMB = (22.5 * Weight) + 499
        elif Age >= 11 and Age <= 18:
            TMB = (12.2 * Weight) + 746
        elif Age >= 19 and Age <= 30:
            TMB = (14.7 * Weight) + 496
        elif Age >= 31 and Age <= 60:
            TMB = (8.7 * Weight) + 829
        else: # edad > 60
            TMB = (10.5 * Weight) + 596

    return TMB

In [106]:
def calcular_TMB_harris_benedict_simplificada(Weight, Height, Age, Gender, PhysicalActivity):
    if Gender == 'MALE':
        partial_TMB = 66.5 + (13.75 * Weight) + (5.003 * Height) - (6.78 * Age)
    else:
        partial_TMB = 655 + (9.56 * Weight) + (1.85 * Height) - (4.68 * Age)
  # Multiplicador según el nivel de actividad
    multiplicador = {
      'No': 1.2,
      'Slight': 1.375,
      'Moderate': 1.55,
      'Strong': 1.725,
      'Very_Strong': 1.9
    }

    # Calorías necesarias según el nivel de actividad
    TMB = partial_TMB * multiplicador[PhysicalActivity]

    return TMB

In [113]:
def calcular_clasificacion_IMC(Weight, Height):
    Height_meters = Height / 100  # Convertir altura a metros
    IMC = Weight / (Height_meters ** 2)

    if IMC < 18.5:
        classificationIMC = 'Insufficient_Weight'
    elif IMC <= 24.9:
        classificationIMC = 'Normal_Weight'
    elif IMC <= 29.9:
        classificationIMC = 'Overweight'
    elif IMC <= 34.9:
        classificationIMC = 'Obesity_Type_I'
    elif IMC <= 39.9:
        classificationIMC = 'Obesity_Type_II'
    else:
        classificationIMC = 'Obesity_Type_III'

    return classificationIMC


In [114]:
def calcular_TMB(Weight, Height, Age, Gender, PhysicalActivity):
  classificationIMC = calcular_clasificacion_IMC(Weight, Height)

  if classificationIMC in ['Insufficient_Weight', 'Normal_Weight']:
        return -1
  elif classificationIMC in ['Obesity_Type_II', 'Obesity_Type_III']:
        return calcular_TMB_MIFFLIN(Weight, Height, Age, Gender, PhysicalActivity)
  else:
      if PhysicalActivity == 'No':
          return calcular_TMB_FAO_ONU(Weight, Age, Gender)
      else:
          return calcular_TMB_harris_benedict_simplificada(Weight, Height, Age, Gender, PhysicalActivity)

In [115]:
def calcular_meta_mensual(pesoActual):
  return pesoActual * 0.02

In [116]:
def calcular_deficit(TMB):
  if TMB == -1:
      return -1
  else:
    deficit = TMB - (TMB * 0.20)
  return deficit

In [117]:
def calcular_macronutrientes(deficit, Diabetes):
    if deficit == -1:
      return "NO DISPONIBLE"
    else:
      c_carbohidratos = deficit * 0.50
      c_grasas = deficit * 0.30

      # Para diabetes, ajustamos la cantidad de proteína según el total calórico
      if Diabetes == 'YES':
          # Ajustamos directamente los gramos de proteínas basándonos en el total calórico
          g_proteinas = deficit * 0.2 / 4 / 0.8
      else:
          c_proteinas = deficit * 0.20
          g_proteinas = c_proteinas / 4

      g_carbohidratos = c_carbohidratos / 4
      g_grasas = c_grasas / 9

      return {
          "Carbohidratos (g)": round(g_carbohidratos, 2),
          "Proteínas (g)": round(g_proteinas, 2),
          "Grasas (g)": round(g_grasas, 2)
      }



In [118]:
train['TMB'] = train.apply(lambda row: calcular_TMB(
    row['Weight'],
    row['Height'],
    row['Age'],
    row['Gender'],
    row['PhysicalActivity']
    ), axis=1)

# Asumiendo que 'train' es tu DataFrame
train['ClassificationIMC'] = train.apply(lambda row: calcular_clasificacion_IMC(row['Weight'], row['Height']), axis=1)

# Calcular el déficit para cada fila y crear la columna 'Deficit'
train['Deficit'] = train['TMB'].apply(calcular_deficit)

def get_macronutrientes(row):
    deficit = row['Deficit']
    diabetes_status = row['Diabetes']  # Asume que 'Diabetes' es el nombre de tu columna
    return calcular_macronutrientes(deficit, diabetes_status) if deficit != -1 else {"Carbohidratos (g)": None, "Proteínas (g)": None, "Grasas (g)": None}

# Asumiendo que 'train' es tu DataFrame y ya tiene las columnas 'Deficit' y 'Diabetes'
macronutrientes = train.apply(get_macronutrientes, axis=1)

# Convertir el diccionario de resultados en un DataFrame y unirlo al DataFrame original
macronutrientes_df = pd.DataFrame(macronutrientes.tolist(), index=train.index)
# Verifica si las columnas ya existen y, de ser así, elimínalas
for col in ['Carbohidratos (g)', 'Proteínas (g)', 'Grasas (g)']:
    if col in train.columns:
        train.drop(col, axis=1, inplace=True)

# Ahora puedes unir sin problemas
train = train.join(macronutrientes_df)

# Asumiendo que tienes un DataFrame llamado 'train' y una columna 'Weight' con el peso actual de la persona en kilogramos
train['PesoAperder'] = train['Weight'].apply(calcular_meta_mensual)


In [119]:
train.head(60)

,Gender,Age,Diabetes,Weight,Height,AbdominalCircumference,PhysicalActivity,TMB,Deficit,PesoAperder,ClassificationIMC,Carbohidratos (g),Proteínas (g),Grasas (g)
0,MALE,39,NO,75.0,171.0,95.6,No,1749.000000,1399.200000,1.500,Overweight,174.90,69.96,46.64
1,FEMALE,22,NO,65.9,158.1,89.5,Slight,2027.477375,1621.981900,1.318,Overweight,202.75,81.10,54.07
2,FEMALE,20,NO,58.3,161.5,82.9,Slight,-1.000000,-1.000000,1.166,Normal_Weight,NaN,NaN,NaN
3,MALE,33,NO,73.8,170.7,93.7,No,1735.080000,1388.064000,1.476,Overweight,173.51,69.40,46.27
4,FEMALE,38,NO,53.9,151.5,74.9,No,-1.000000,-1.000000,1.078,Normal_Weight,NaN,NaN,NaN
5,MALE,35,NO,90.3,176.1,103.2,No,1926.480000,1541.184000,1.806,Overweight,192.65,77.06,51.37
6,FEMALE,26,NO,62.4,156.1,84.5,Moderate,2198.905950,1759.124760,1.248,Overweight,219.89,87.96,58.64
7,FEMALE,24,NO,68.3,155.7,89.2,Moderate,2299.693150,1839.754520,1.366,Overweight,229.97,91.99,61.33
8,MALE,34,NO,81.5,169.0,98.0,No,1824.400000,1459.520000,1.630,Overweight,182.44,72.98,48.65
9,MALE,34,NO,52.9,164.6,71.5,Slight,-1.000000,-1.000000,1.058,Normal_Weight,NaN,NaN,NaN


carbohidratos = 187.80
proteinas = 75.12
grasas = 50.08

# Desayuno (25%)
desayuno_carbs = carbohidratos * 0.25
desayuno_prot = proteinas * 0.25
desayuno_grasas = grasas * 0.25

# Almuerzo (35%)
almuerzo_carbs = carbohidratos * 0.35
almuerzo_prot = proteinas * 0.35
almuerzo_grasas = grasas * 0.35

# Cena (40%)
cena_carbs = carbohidratos * 0.40
cena_prot = proteinas * 0.40
cena_grasas = grasas * 0.40

# Resultados
(desayuno_carbs, desayuno_prot, desayuno_grasas), (almuerzo_carbs, almuerzo_prot, almuerzo_grasas), (cena_carbs, cena_prot, cena_grasas)

In [ ]:
# Cargar el nuevo dataset para revisar su contenido
new_file_path = '/nutrients_csvfile.csv'
food_dataset = pd.read_csv(new_file_path)

# Mostrar las primeras filas del nuevo dataset para entender su estructura
food_dataset.head()

,Food,Measure,Grams,Calories,Protein,Fat,Sat.Fat,Fiber,Carbs,Category
0,Cows' milk,1 qt.,976,660,32,40,36,0,48,Dairy products
1,Milk skim,1 qt.,984,360,36,t,t,0,52,Dairy products
2,Buttermilk,1 cup,246,127,9,5,4,0,13,Dairy products
3,"Evaporated, undiluted",1 cup,252,345,16,20,18,0,24,Dairy products
4,Fortified milk,6 cups,"1,419","1,373",89,42,23,1.4,119,Dairy products


In [ ]:
# Revisar las categorías únicas de alimentos para identificar posibles candidatos a excluir
categorias_unicas_nuevas = food_dataset['Category'].unique()
categorias_unicas_nuevas

array(['Dairy products', 'Fats, Oils, Shortenings', 'Meat, Poultry',
       'Fish, Seafood', 'Vegetables A-E', 'Vegetables F-P',
       'Vegetables R-Z', 'Fruits A-F', 'Fruits G-P', 'Fruits R-Z',
       'Breads, cereals, fastfood,grains', 'Soups', 'Desserts, sweets',
       'Jams, Jellies', 'Seeds and Nuts', 'Drinks,Alcohol, Beverages'],
      dtype=object)

In [ ]:
# Filtrar el dataset para excluir categorías seleccionadas como potencialmente no adecuadas para una dieta balanceada
categorias_a_excluir = ['Desserts, sweets', 'Jams, Jellies', 'Drinks,Alcohol, Beverages']

# Filtrar el dataset para eliminar las filas que pertenecen a las categorías seleccionadas
dataset_filtrado = food_dataset[~food_dataset['Category'].isin(categorias_a_excluir)]

# Mostrar las primeras filas del dataset filtrado para verificar el resultado
dataset_filtrado.head()


,Food,Measure,Grams,Calories,Protein,Fat,Sat.Fat,Fiber,Carbs,Category
0,Cows' milk,1 qt.,976,660,32,40,36,0,48,Dairy products
1,Milk skim,1 qt.,984,360,36,t,t,0,52,Dairy products
2,Buttermilk,1 cup,246,127,9,5,4,0,13,Dairy products
3,"Evaporated, undiluted",1 cup,252,345,16,20,18,0,24,Dairy products
4,Fortified milk,6 cups,"1,419","1,373",89,42,23,1.4,119,Dairy products


In [ ]:
# Cargar el dataset filtrado previamente exportado para aplicar el nuevo filtro
filtrado = pd.read_csv('/nutrients_filtered.csv')

# Convertir las columnas relevantes a numérico, manejando posibles valores no numéricos
filtrado['Carbs'] = pd.to_numeric(filtrado['Carbs'], errors='coerce')
filtrado['Fiber'] = pd.to_numeric(filtrado['Fiber'], errors='coerce')

# Definir el criterio para filtrar: alimentos con más de 20g de carbohidratos y menos de 2g de fibra por 100g
criterio_carbs_altos_fibra_baja = (filtrado['Carbs'] > 20) & (filtrado['Fiber'] < 2)

# Aplicar el filtro para excluir estos alimentos
dataset_filtrado_refinado = filtrado[~criterio_carbs_altos_fibra_baja]

# Mostrar las primeras filas del dataset refinado para verificar el resultado
dataset_filtrado_refinado.head()


,Food,Measure,Grams,Calories,Protein,Fat,Sat.Fat,Fiber,Carbs,Category
2,Buttermilk,1 cup,246,127,9,5,4,0.0,13.0,Dairy products
8,Goats' milk,1 cup,244,165,8,10,8,0.0,11.0,Dairy products
11,skim. milk,1 cup,250,128,18,4,3,1.0,13.0,Dairy products
16,Cream or half-and-half,1/2 cup,120,170,4,15,13,0.0,5.0,Dairy products
17,or whipping,1/2 cup,119,430,2,44,27,1.0,3.0,Dairy products
